In [63]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import math
import numpy as np

In [64]:
df = pd.read_csv('processed_data/11-pools.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98660 entries, 0 to 98659
Data columns (total 95 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         98660 non-null  uint64 
 1   org_id                          98660 non-null  uint64 
 2   rating                          98660 non-null  float64
 3   ts                              98660 non-null  int64  
 4   user_city                       98660 non-null  object 
 5   org_city                        98660 non-null  object 
 6   average_bill                    98660 non-null  float64
 7   rating_org                      98660 non-null  float64
 8   rubrics                         98660 non-null  object 
 9   food_delivery                   98660 non-null  int64  
 10  breakfast                       98660 non-null  int64  
 11  takeaway                        98660 non-null  int64  
 12  summer_terrace                  

In [65]:
df.describe()

,user_id,org_id,rating,ts,average_bill,rating_org,food_delivery,breakfast,takeaway,summer_terrace,...,is_restraunt,is_caffe,is_other,resist_covid,for_adult,rating_1,rating_2,rating_3,rating_4,rating_5
count,9.866000e+04,9.866000e+04,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,...,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000,98660.000000
mean,9.196190e+18,9.050700e+18,4.322481,715.047172,1086.206665,3.239499,0.758504,0.577509,0.650760,0.603061,...,0.357511,0.255544,0.386945,0.784827,0.205777,0.058281,0.031573,0.084817,0.180043,0.645287
std,5.315843e+18,5.272360e+18,1.130019,293.314486,2403.581646,1.889331,0.427992,0.493958,0.476732,0.489266,...,0.479269,0.436169,0.487053,0.410945,0.404271,0.234275,0.174861,0.278610,0.384225,0.478429
min,1.445171e+14,1.626686e+15,1.000000,0.000000,500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.604080e+18,4.540661e+18,4.000000,505.000000,500.000000,0.750000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.203901e+18,9.104453e+18,5.000000,699.000000,1000.000000,4.256959,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.377072e+19,1.351377e+19,5.000000,958.000000,1173.467919,4.476636,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.844666e+19,1.844622e+19,5.000000,1216.000000,250000.000000,5.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# ml

In [66]:
# df.columns
columns = [
  # 'user_id', 
  # 'org_id', 
  # 'rating', 
  # 'ts', 
  'user_city', 
  'org_city',
  # 'average_bill', 
  'rating_org', # target
  # 'rubrics', 
  'food_delivery', 
  'breakfast',
  'takeaway', 
  'summer_terrace', 
  'wi_fi', 
  'business_lunch',
  'payment_by_credit_card', 
  'coffee_to_go', 
  'closed_for_quarantine',
  'online_takeaway', 
  'karaoke', 
  'special_menu', 
  'sports_broadcasts',
  'food_court1', 
  'craft_beer', 
  'projector', 
  'gift_certificate',
  'wheelchair_access', 
  'automatic_door', 
  'food_product_delivery',
  'car_park', 
  'parking_disabled', 
  'toilet_for_disabled', 
  'promotions',
  'cafe', 
  'minimum_order', 
  'free_delivery', 
  'self_service_kiosks',
  'has_restaurant', 
  'air_conditioning', 
  'pets', 
  'elevator', 
  'has_bar', 
  'internet access', 
  'privilege_tickets', 
  'organic_food', 
  'vip_zone',
  'hall_capacity', 
  'face_control',  
  'show_program', 
  'music',
  'chillout',
  'dancefloor', 
  'dress_code', 
  'billiards', 
  'strip', 
  'elevator_wheelchair_accessible', 
  'ramp', 
  'around_the_clock_work1',
  'pickup', 
  'farm_products', 
  'delivery', 
  'coffee_shop', 
  'call_button',
  'wheelchair_accessible', 
  'table_games', 
  'vinotheque', 
  'cakes_for_different_occasions',
  'accepted_credit_cards', 
  'kalyan', 
  'teahouse',
  'bread_from_tandoor',
  'handmade_goods', 

  'dateMM',
  'bill500', 
  'bill1000',
  'bill1500',
  'bill2000',
  'bill2500',
  'bill3000',
  'bill3500',
  'bill4000',
  'billExpensive', 
  # 'native_citizen', 
  # 'tourist', 
  'is_restraunt',
  'is_caffe', 
  'is_other',
  # 'resist_covid',
  # 'for_adult',
  "rating_1",
  "rating_2",
  "rating_3",
  "rating_4",
  "rating_5",
  'pool'
]

df1 = df[columns]
df1 = pd.get_dummies(df1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98660 entries, 0 to 98659
Columns: 128 entries, rating_org to pool_train
dtypes: float64(1), int64(80), uint8(47)
memory usage: 65.4 MB


In [67]:
print(df1.shape)
df1.head(5)

(98660, 128)


,rating_org,food_delivery,breakfast,takeaway,summer_terrace,wi_fi,business_lunch,payment_by_credit_card,coffee_to_go,closed_for_quarantine,...,dateMM_M39,dateMM_M4,dateMM_M40,dateMM_M5,dateMM_M6,dateMM_M7,dateMM_M8,dateMM_M9,pool_test,pool_train
0,3.883212,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,4.495379,1,0,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
2,4.576063,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,3.775701,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
4,4.187500,1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1


In [68]:
for col in df1.columns.values:
  if col.startswith('dateMM'):
    df1[col] = df1[col] * 1000
  if col in ['user_city_msk', 'user_city_spb', 'org_city_msk',
       'org_city_spb']:
    df1[col] = df1[col] * 100
  if col.startswith('bill'):
    df1[col] = df1[col] * 200
  if col.startswith('rating_') and col != "rating_org":
    df1[col] = df1[col] * 300
  if (col.startswith('is_')):
    df1[col] = df1[col] * 200


In [72]:
train, test = df1[df1.pool_train == 1], df1[df1.pool_test == 1]
target_column = 'rating_org'

train_x, train_y = train.drop([target_column, 'pool_train', 'pool_test'], axis=1), train[target_column]
test_x = test.drop([target_column, 'pool_train', 'pool_test'], axis=1)

model = KNeighborsRegressor(n_neighbors=300, weights='distance')
model.fit(train_x, train_y)

predict = model.predict(test_x)


In [70]:
# # w = np.asarray(([1.0] * 80) + ([2.0] * 4) + ([5.0] * 41))
# # def dist(a, b):
# #   return math.sqrt(np.sum(np.multiply(w, np.square(a - b))))

# train, test = train_test_split(df1, test_size=0.5)
# target_column = 'rating_org'

# train_x, train_y = train.drop(target_column, axis=1), train[target_column]
# test_x, test_y = test.drop(target_column, axis=1), test[target_column]

# model = KNeighborsRegressor(n_neighbors=500, weights='distance', metric=dist)
# model.fit(train_x, train_y)

# predict = model.predict(test_x)
# print('MAPE:', mean_absolute_percentage_error(test_y, predict))

In [73]:
result = pd.DataFrame(data=predict, columns=['rating_org'])
result.index.names = ['id']

result.to_csv('result/better_distrib_old_2_features.csv')